In [3]:
pip install numpy


Note: you may need to restart the kernel to use updated packages.


In [5]:
pip install pandas

   ---------------------------------------- 0.0/11.5 MB ? eta -:--:--
   -- ------------------------------------- 0.8/11.5 MB 6.8 MB/s eta 0:00:02
   --------------------- ------------------ 6.0/11.5 MB 18.3 MB/s eta 0:00:01
   ---------------------------------------  11.3/11.5 MB 21.9 MB/s eta 0:00:01
   ---------------------------------------- 11.5/11.5 MB 17.7 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd

In [14]:
df=pd.read_csv(r'C:\Users\ADMIN\Downloads\spam.csv',encoding='latin-1')
df=df[['v1','v2']]
df.columns=['category','text']

In [16]:
df['category'] = df['category'].map({'ham': 0, 'spam': 1})


In [17]:
pip install nltk


   ---------------------------------------- 0.0/1.5 MB ? eta -:--:--
   -------------------- ------------------- 0.8/1.5 MB 5.0 MB/s eta 0:00:01
   ---------------------------------------- 1.5/1.5 MB 4.9 MB/s eta 0:00:00


In [18]:
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
import re

nltk.download('stopwords')
stop_words = set(stopwords.words('english'))
stemmer = PorterStemmer()

def preprocess_text(text):
    text = re.sub(r'\W', ' ', text)  # Remove special characters
    text = text.lower()             # Convert to lowercase
    text = text.split()             # Tokenize
    text = [stemmer.stem(word) for word in text if word not in stop_words]
    return ' '.join(text)

df['processed_text'] = df['text'].apply(preprocess_text)


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ADMIN\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


In [24]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer(max_features=3000)  # Use the top 3000 features
X = tfidf.fit_transform(df['processed_text']).toarray()
y = df['category']


In [22]:
pip install scikit-learn

   ---------------------------------------- 0.0/11.1 MB ? eta -:--:--
   --- ------------------------------------ 1.0/11.1 MB 9.9 MB/s eta 0:00:02
   ---------------------- ----------------- 6.3/11.1 MB 19.1 MB/s eta 0:00:01
   ---------------------------------------  11.0/11.1 MB 21.1 MB/s eta 0:00:01
   ---------------------------------------- 11.1/11.1 MB 18.0 MB/s eta 0:00:00
   ---------------------------------------- 0.0/44.5 MB ? eta -:--:--
   ------ --------------------------------- 7.1/44.5 MB 34.8 MB/s eta 0:00:02
   --------------- ------------------------ 16.8/44.5 MB 35.8 MB/s eta 0:00:01
   ----------------------- ---------------- 26.2/44.5 MB 37.2 MB/s eta 0:00:01
   -------------------------------- ------- 36.4/44.5 MB 38.8 MB/s eta 0:00:01
   ---------------------------------------  44.3/44.5 MB 40.3 MB/s eta 0:00:01
   ---------------------------------------  44.3/44.5 MB 40.3 MB/s eta 0:00:01
   ---------------------------------------- 44.5/44.5 MB 29.7 MB/s eta 0:0

In [25]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [26]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

model = LogisticRegression()
model.fit(X_train, y_train)

# Predictions
y_pred = model.predict(X_test)

# Evaluation
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred))


Accuracy: 0.9605381165919282
Classification Report:
               precision    recall  f1-score   support

           0       0.96      1.00      0.98       959
           1       0.97      0.74      0.84       156

    accuracy                           0.96      1115
   macro avg       0.97      0.87      0.91      1115
weighted avg       0.96      0.96      0.96      1115



In [27]:
pip install pickle

Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement pickle (from versions: none)
ERROR: No matching distribution found for pickle


In [28]:
import pickle
with open('spam_detector.pkl', 'wb') as f:
    pickle.dump(model, f)
with open('tfidf_vectorizer.pkl', 'wb') as f:
    pickle.dump(tfidf, f)


In [31]:
with open('spam_detector.pkl', 'rb') as f:
    loaded_model = pickle.load(f)
with open('tfidf_vectorizer.pkl', 'rb') as f:
    loaded_tfidf = pickle.load(f)

sample_text = input("Enter text: ");
sample_text = preprocess_text(sample_text)
sample_vector = loaded_tfidf.transform([sample_text]).toarray()
prediction = loaded_model.predict(sample_vector)
print("Spam" if prediction[0] == 1 else "Not Spam")


Enter text:  This is Mr. Ramareddy from CBI! It's observed that your credit card has been used in illegal operations.


Not Spam
